In [1]:
import requests
from utils import DayCET
import json
import os
import pandas as pd
from tqdm import tqdm

In [2]:
rte_id_64 = "ODRlNjNhN2QtOTA4NC00YzZmLWFlMzQtNWYwOTg5YmRkNzI3OjVlNjg5NjE3LTg1MDItNDM2MC04YzE5LWFkNzlhYjk1OWY2Zg=="
ROOT_PATH = r'C:\Users\Raphaël Thireau\OneDrive - versoenergy\Documents\datapipeline'

In [3]:
def get_token_and_type(id = rte_id_64):
    url = "https://digital.iservices.rte-france.com/token/oauth/"
    headers = {"Authorization": f"Basic {id}"}
    response = requests.post(url, headers=headers)
    if response.status_code == 200:
        resp_dict = response.json()
        return resp_dict['access_token'], resp_dict['token_type']
    else:
        print(f"Error: {response.status_code}, {response.text}")

In [7]:
def get_rte_conso(day : DayCET, RTE_token = None, id = rte_id_64):
    if RTE_token is None:
        token, token_type = get_token_and_type(id=id)
    else:
        token, token_type = RTE_token
    start = day.start_utc.tz_localize('UTC').isoformat().replace('+', '%2B')
    end = day.end_utc.tz_localize('UTC').isoformat().replace('+', '%2B')
    url = f'https://digital.iservices.rte-france.com/open_api/consumption/v1/short_term?start_date={start}&end_date={end}'
    headers = {"Authorization": f"{token_type} {token}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(os.path.join(ROOT_PATH, 'process', 'data', 'RTE', 'Consumption', 'FR', f'{day.date_str.replace('-', '')}_RTE_Cons_FR.json'), 'w') as file:
            json.dump(response.json(), file, indent=4)
    else:
        raise Exception(f"Error: {response.status_code}, {response.text}")

In [8]:
RTE_tocken = get_token_and_type()
for date in tqdm(pd.date_range(start='2025-01-09', end='2025-01-26', freq='D', inclusive='both')):
    get_rte_conso(day=DayCET(date.strftime(format='%Y-%m-%d')), RTE_token=RTE_tocken)

100%|██████████| 18/18 [00:03<00:00,  4.86it/s]


In [9]:
def get_rte_prod(day : DayCET, RTE_token = None, id = rte_id_64):
    if RTE_token is None:
        token, token_type = get_token_and_type(id=id)
    else:
        token, token_type = RTE_token
    start = day.start_utc.tz_localize('UTC').isoformat().replace('+', '%2B')
    end = day.end_utc.tz_localize('UTC').isoformat().replace('+', '%2B')
    url = f'https://digital.iservices.rte-france.com/open_api/actual_generation/v1/actual_generations_per_production_type?start_date={start}&end_date={end}'
    headers = {"Authorization": f"{token_type} {token}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(os.path.join(ROOT_PATH, 'process', 'data', 'RTE', 'Production', 'FR', f'{day.date_str.replace('-', '')}_RTE_Prod_FR.json'), 'w') as file:
            json.dump(response.json(), file, indent=4)
    else:
        raise Exception(f"Error: {response.status_code}, {response.text}")

In [10]:
RTE_tocken = get_token_and_type()
for date in tqdm(pd.date_range(start='2025-01-09', end='2025-01-26', freq='D', inclusive='both')):
    get_rte_prod(day=DayCET(date.strftime(format='%Y-%m-%d')), RTE_token=RTE_tocken)

100%|██████████| 18/18 [00:02<00:00,  6.01it/s]


In [50]:
def get_rte_capa(year, RTE_token = None, id = rte_id_64):
    if RTE_token is None:
        token, token_type = get_token_and_type(id=id)
    else:
        token, token_type = RTE_token
    start = pd.Timestamp(year=year, month=1, day=1, tz='CET').isoformat().replace('+', '%2B')
    end = pd.Timestamp(year=year+1, month=1, day=1, tz='CET').isoformat().replace('+', '%2B')
    
    url = f'https://digital.iservices.rte-france.com/open_api/generation_installed_capacities/v1/capacities_per_production_type?start_date={start}&end_date={end}'
    headers = {"Authorization": f"{token_type} {token}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(os.path.join(ROOT_PATH, 'process', 'data', 'RTE', 'Capacities', 'FR', f'{year}_RTE_Capa_FR.json'), 'w') as file:
            json.dump(response.json(), file, indent=4)
    else:
        raise Exception(f"Error: {response.status_code}, {response.text}")

In [53]:
RTE_tocken = get_token_and_type()
for year in range(2015, 2025):
    get_rte_capa(year=year, RTE_token=RTE_tocken)